# Classification models - Supervised Discretization
## Dataset: pima

Updated by: Sam
Update at: 13/10/2022 <br>
---
Discretization methods: ChiMerge (manually defined function)<br>
Classification models: CNB, ID3, KNN-VDM
---
NOTE: 
NEED TO DO SMOTE for imbalance dataset before training
Long time for computation of Knn-VDM (Run this part last) <br>
Use Malina scripts for Knn, ID3 <br>
Use Sam scripts (with min_categories) for Naive Bayes <br>
Key Errors for decomposition KNN-VDM models k = 15

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import read_csv
from pandas import set_option
import numpy as np
from numpy import arange
## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#read in data from ChiMerge discretization
pima6 = pd.read_csv('chim_pima_6int.csv')
pima8 = pd.read_csv('chim_pima_8int.csv')
pima10 = pd.read_csv('chim_pima_10int.csv')
pima15 = pd.read_csv('chim_pima_15int.csv')

In [5]:
pima6.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,3,2,3,0,4,2,3,1
1,1,1,2,2,0,1,2,2,0
2,3,5,2,0,0,0,2,2,1
3,1,1,2,1,2,1,0,0,0
4,0,3,0,3,5,4,5,2,1


## Interval frequencies

In [6]:
num_list6 = list(pima6.drop('Outcome', axis=1).columns)
num_list8 = list(pima8.drop('Outcome', axis=1).columns)
num_list10 = list(pima10.drop('Outcome', axis=1).columns)
num_list15 = list(pima15.drop('Outcome', axis=1).columns)

In [7]:
print('Interval frequency for 6 Intervals')
for i in num_list6:
    print('Interval for ' + i)
    print(Counter(pima6[i]))

Interval frequency for 6 Intervals
Interval for Pregnancies
Counter({2: 250, 1: 238, 3: 111, 0: 111, 4: 54, 5: 4})
Interval for Glucose
Counter({2: 288, 1: 192, 3: 161, 5: 79, 4: 43, 0: 5})
Interval for BloodPressure
Counter({2: 320, 3: 276, 1: 119, 0: 40, 5: 10, 4: 3})
Interval for SkinThickness
Counter({0: 229, 3: 211, 1: 170, 2: 150, 5: 4, 4: 4})
Interval for Insulin
Counter({0: 375, 5: 221, 1: 117, 4: 29, 2: 22, 3: 4})
Interval for BMI
Counter({4: 315, 1: 253, 0: 167, 5: 14, 3: 10, 2: 9})
Interval for DiabetesPedigreeFunction
Counter({2: 467, 0: 265, 3: 18, 4: 9, 5: 5, 1: 4})
Interval for Age
Counter({0: 219, 1: 198, 2: 197, 3: 100, 4: 33, 5: 21})


In [8]:
print('Frequency for aus with 8 Intervals')
for i in num_list8:
    print('Interval for ' + i)
    print(Counter(pima8[i]))

Frequency for aus with 8 Intervals
Interval for Pregnancies
Counter({2: 250, 1: 238, 3: 111, 0: 111, 4: 24, 6: 19, 5: 11, 7: 4})
Interval for Glucose
Counter({1: 192, 2: 166, 5: 161, 4: 112, 7: 79, 6: 43, 3: 10, 0: 5})
Interval for BloodPressure
Counter({4: 320, 5: 276, 3: 73, 0: 40, 2: 40, 7: 10, 1: 6, 6: 3})
Interval for SkinThickness
Counter({0: 229, 3: 207, 1: 170, 2: 150, 7: 4, 6: 4, 4: 3, 5: 1})
Interval for Insulin
Counter({0: 375, 7: 221, 1: 104, 6: 29, 2: 13, 4: 12, 3: 10, 5: 4})
Interval for BMI
Counter({4: 298, 1: 253, 0: 167, 7: 14, 6: 11, 3: 10, 2: 9, 5: 6})
Interval for DiabetesPedigreeFunction
Counter({3: 247, 1: 243, 4: 220, 0: 22, 5: 18, 6: 9, 7: 5, 2: 4})
Interval for Age
Counter({2: 198, 3: 197, 1: 156, 4: 100, 0: 63, 5: 33, 7: 13, 6: 8})


In [9]:
print('Frequency for aus with 10 Intervals')
for i in num_list10:
    print('Interval for ' + i)
    print(Counter(pima10[i]))

Frequency for aus with 10 Intervals
Interval for Pregnancies
Counter({3: 250, 1: 135, 0: 111, 2: 103, 4: 83, 5: 28, 6: 24, 8: 19, 7: 11, 9: 4})
Interval for Glucose
Counter({1: 192, 2: 166, 7: 161, 9: 79, 4: 73, 8: 43, 5: 25, 6: 14, 3: 10, 0: 5})
Interval for BloodPressure
Counter({7: 276, 4: 159, 5: 153, 3: 73, 0: 40, 2: 40, 9: 10, 6: 8, 1: 6, 8: 3})
Interval for SkinThickness
Counter({0: 229, 4: 204, 3: 150, 2: 139, 1: 31, 9: 4, 8: 4, 6: 3, 5: 3, 7: 1})
Interval for Insulin
Counter({0: 375, 9: 221, 1: 56, 2: 46, 8: 29, 4: 13, 6: 12, 5: 10, 7: 4, 3: 2})
Interval for BMI
Counter({4: 268, 1: 253, 0: 167, 5: 24, 9: 14, 8: 11, 3: 10, 2: 9, 7: 6, 6: 6})
Interval for DiabetesPedigreeFunction
Counter({4: 247, 2: 237, 6: 214, 0: 22, 7: 18, 8: 9, 5: 6, 1: 6, 9: 5, 3: 4})
Interval for Age
Counter({2: 198, 3: 197, 1: 156, 0: 63, 5: 47, 6: 40, 7: 33, 4: 13, 9: 13, 8: 8})


In [10]:
print('Frequency for aus with 15 Intervals')
for i in num_list15:
    print('Interval for ' + i)
    print(Counter(pima15[i]))

Frequency for aus with 15 Intervals
Interval for Pregnancies
Counter({1: 135, 0: 111, 2: 103, 3: 75, 4: 68, 5: 57, 6: 50, 7: 45, 8: 38, 9: 28, 10: 24, 11: 11, 13: 10, 12: 9, 14: 4})
Interval for Glucose
Counter({3: 166, 2: 163, 8: 153, 14: 79, 5: 73, 1: 29, 12: 26, 6: 25, 7: 14, 4: 10, 10: 8, 9: 8, 13: 7, 0: 5, 11: 2})
Interval for BloodPressure
Counter({6: 159, 7: 153, 11: 152, 10: 85, 3: 40, 0: 39, 9: 39, 5: 38, 4: 35, 13: 9, 8: 8, 2: 6, 12: 3, 1: 1, 14: 1})
Interval for SkinThickness
Counter({0: 229, 5: 150, 6: 145, 4: 113, 7: 59, 1: 31, 3: 20, 2: 6, 11: 4, 9: 3, 8: 3, 14: 2, 13: 1, 10: 1, 12: 1})
Interval for Insulin
Counter({0: 375, 14: 163, 12: 48, 5: 44, 11: 29, 1: 28, 3: 25, 7: 13, 9: 12, 8: 10, 13: 10, 10: 4, 2: 3, 4: 2, 6: 2})
Interval for BMI
Counter({9: 268, 6: 166, 4: 91, 5: 87, 1: 48, 10: 24, 14: 14, 0: 11, 2: 11, 13: 11, 8: 10, 7: 9, 3: 6, 12: 6, 11: 6})
Interval for DiabetesPedigreeFunction
Counter({11: 214, 9: 211, 2: 159, 4: 74, 6: 29, 0: 22, 12: 18, 13: 9, 10: 6, 1: 

In [11]:
pima6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Pregnancies               768 non-null    int64
 1   Glucose                   768 non-null    int64
 2   BloodPressure             768 non-null    int64
 3   SkinThickness             768 non-null    int64
 4   Insulin                   768 non-null    int64
 5   BMI                       768 non-null    int64
 6   DiabetesPedigreeFunction  768 non-null    int64
 7   Age                       768 non-null    int64
 8   Outcome                   768 non-null    int64
dtypes: int64(9)
memory usage: 54.1 KB


# 1. Categorical Naive Bayes

## 1.1 CNB, 6 Intervals from ChiMerge

In [14]:
# make test & train split
X = pima6.drop('Outcome', axis=1)
y = pima6['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [15]:
# Check representation of class
print('Class representation - original: ', Counter(pima6['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [16]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [17]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [19]:
# SAM
# Count number of categories
n_categories = pima6.drop('Outcome', axis=1).nunique()

In [20]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 47
Accuracy: 0.76
Recall score :  0.7552083333333334
Precision score :  0.7552083333333334
F1 score :  0.7552083333333334
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.66      0.78       125
           1       0.59      0.94      0.73        67

    accuracy                           0.76       192
   macro avg       0.77      0.80      0.75       192
weighted avg       0.83      0.76      0.76       192

Computation time:
0.01199960708618164


In [21]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.248
Average bias: 0.250
Average variance: 0.037
Sklearn 0-1 loss: 0.245


## 1.2 CNB, 8 Intervals from ChiMerge

In [22]:
# make test & train split
X = pima8.drop('Outcome', axis=1)
y = pima8['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [23]:
# Check representation of class
print('Class representation - original: ', Counter(pima8['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [24]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [25]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [26]:
# SAM
# Count number of categories
n_categories = pima8.drop('Outcome', axis=1).nunique()

In [27]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 46
Accuracy: 0.76
Recall score :  0.7604166666666666
Precision score :  0.7604166666666666
F1 score :  0.7604166666666666
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.67      0.79       125
           1       0.60      0.93      0.73        67

    accuracy                           0.76       192
   macro avg       0.77      0.80      0.76       192
weighted avg       0.82      0.76      0.77       192

Computation time:
0.012999296188354492


In [28]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.251
Average bias: 0.250
Average variance: 0.049
Sklearn 0-1 loss: 0.240


## 1.3 CNB, 10 Intervals from ChiMerge

In [29]:
# make test & train split
X = pima10.drop('Outcome', axis=1)
y = pima10['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [30]:
# Check representation of class
print('Class representation - original: ', Counter(pima10['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [31]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [32]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [34]:
# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

Class representation - training data:  Counter({1: 375, 0: 375})


In [36]:
# SAM
# Count number of categories
n_categories = pima10.drop('Outcome', axis=1).nunique()

In [37]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 48
Accuracy: 0.75
Recall score :  0.75
Precision score :  0.75
F1 score :  0.75
Classification report:
              precision    recall  f1-score   support

           0       0.91      0.68      0.78       125
           1       0.60      0.88      0.71        67

    accuracy                           0.75       192
   macro avg       0.75      0.78      0.75       192
weighted avg       0.80      0.75      0.76       192

Computation time:
0.012998342514038086


In [38]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.256
Average bias: 0.250
Average variance: 0.061
Sklearn 0-1 loss: 0.250


## 1.4 CNB, 15 Intervals from ChiMerge

In [39]:
# make test & train split
X = pima15.drop('Outcome', axis=1)
y = pima15['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [40]:
# Check representation of class
print('Class representation - original: ', Counter(pima15['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [41]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [42]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [43]:
# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

Class representation - training data:  Counter({1: 375, 0: 375})


In [44]:
# SAM
# Count number of categories
n_categories = pima15.drop('Outcome', axis=1).nunique()

In [45]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 50
Accuracy: 0.74
Recall score :  0.7395833333333334
Precision score :  0.7395833333333334
F1 score :  0.7395833333333334
Classification report:
              precision    recall  f1-score   support

           0       0.89      0.69      0.77       125
           1       0.59      0.84      0.69        67

    accuracy                           0.74       192
   macro avg       0.74      0.76      0.73       192
weighted avg       0.78      0.74      0.75       192

Computation time:
0.01299905776977539


In [46]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.251
Average bias: 0.255
Average variance: 0.069
Sklearn 0-1 loss: 0.260


# 2. Decision Tree ID3

## 2.1 ID3, 6 Intervals from ChiMerge

In [47]:
# Prepare data and apply SMOTE
# make test & train split
X = pima6.drop('Outcome', axis=1)
y = pima6['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima6['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima6.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [48]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.69
Recall score :  0.6927083333333334
Precision score :  0.6927083333333334
F1 score :  0.6927083333333334
Classification report:
              precision    recall  f1-score   support

           0       0.80      0.70      0.75       125
           1       0.55      0.67      0.60        67

    accuracy                           0.69       192
   macro avg       0.67      0.69      0.68       192
weighted avg       0.71      0.69      0.70       192

Computation time:
0.057999610900878906


In [49]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.328
Average bias: 0.281
Average variance: 0.201
Sklearn 0-1 loss: 0.307


## 2.2 ID3, 8 Intervals from ChiMerge

In [50]:
# Prepare data and apply SMOTE
# make test & train split
X = pima8.drop('Outcome', axis=1)
y = pima8['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima8['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima8.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [51]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.67
Recall score :  0.6666666666666666
Precision score :  0.6666666666666666
F1 score :  0.6666666666666666
Classification report:
              precision    recall  f1-score   support

           0       0.82      0.62      0.71       125
           1       0.52      0.75      0.61        67

    accuracy                           0.67       192
   macro avg       0.67      0.69      0.66       192
weighted avg       0.71      0.67      0.67       192

Computation time:
0.06553030014038086


In [52]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.338
Average bias: 0.323
Average variance: 0.180
Sklearn 0-1 loss: 0.333


## 2.3 ID3, 10 Intervals from ChiMerge

In [53]:
# Prepare data and apply SMOTE
# make test & train split
X = pima10.drop('Outcome', axis=1)
y = pima10['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima10['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima10.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [54]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.69
Recall score :  0.6875
Precision score :  0.6875
F1 score :  0.6875
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.68      0.74       125
           1       0.54      0.70      0.61        67

    accuracy                           0.69       192
   macro avg       0.67      0.69      0.67       192
weighted avg       0.72      0.69      0.69       192

Computation time:
0.07196903228759766


In [55]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.327
Average bias: 0.297
Average variance: 0.179
Sklearn 0-1 loss: 0.312


## 2.4 ID3, 15 Intervals from CHiMerge

In [56]:
# Prepare data and apply SMOTE
# make test & train split
X = pima15.drop('Outcome', axis=1)
y = pima15['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima15['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima15.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [57]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.65
Recall score :  0.6458333333333334
Precision score :  0.6458333333333334
F1 score :  0.6458333333333334
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.57      0.68       125
           1       0.50      0.79      0.61        67

    accuracy                           0.65       192
   macro avg       0.67      0.68      0.64       192
weighted avg       0.72      0.65      0.65       192

Computation time:
0.08424687385559082


In [58]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.331
Average bias: 0.297
Average variance: 0.191
Sklearn 0-1 loss: 0.354


# 3. KNN classification

## 3.1 KNN, 6 Intervals from CHiMerge

In [72]:
# Prepare data and apply SMOTE
# make test & train split
X = pima6.drop('Outcome', axis=1)
y = pima6['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima6['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima6.drop('Outcome', axis=1).nunique()


Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [73]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.93      0.74      0.83       125
           1       0.65      0.90      0.75        67

    accuracy                           0.80       192
   macro avg       0.79      0.82      0.79       192
weighted avg       0.83      0.80      0.80       192

Time for training model Knn-VDM: 44.129278898239136.


In [74]:
X_train.values

array([[2, 2, 3, ..., 4, 0, 3],
       [3, 4, 3, ..., 1, 0, 3],
       [0, 3, 2, ..., 0, 0, 0],
       ...,
       [3, 2, 3, ..., 4, 2, 2],
       [3, 5, 3, ..., 4, 2, 2],
       [1, 5, 2, ..., 1, 0, 2]], dtype=int64)

In [75]:
# !!!! WARING: TIME CONSUMING

# Bias and variance decomposition
# Convert all dataframe to array first
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

Average expected loss: 0.255
Average bias: 0.198
Average variance: 0.131
Sklearn 0-1 loss: 0.203


## 3.2 KNN with ChiMerge 8 intervals

In [76]:
# Prepare data and apply SMOTE
# make test & train split
X = pima8.drop('Outcome', axis=1)
y = pima8['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima8['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima8.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [77]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.88      0.74      0.80       125
           1       0.62      0.82      0.71        67

    accuracy                           0.77       192
   macro avg       0.75      0.78      0.76       192
weighted avg       0.79      0.77      0.77       192

Time for training model Knn-VDM: 37.5856409072876.


In [78]:
# !!!! WARING: TIME CONSUMING

# Bias and variance decomposition
# Convert all dataframe to array first
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

Average expected loss: 0.253
Average bias: 0.234
Average variance: 0.126
Sklearn 0-1 loss: 0.234


## 3.3 KNN with ChiMerge 10 intervals

In [79]:
# Prepare data and apply SMOTE
# make test & train split
X = pima10.drop('Outcome', axis=1)
y = pima10['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima10['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima10.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [80]:
# Knn-VDM complete code
# accuracy                           0.79       192
# Time for training model Knn-VDM: 39.84472727775574.
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.88      0.78      0.83       125
           1       0.66      0.79      0.72        67

    accuracy                           0.79       192
   macro avg       0.77      0.79      0.77       192
weighted avg       0.80      0.79      0.79       192

Time for training model Knn-VDM: 39.26955699920654.


In [81]:
# !!!! WARING: TIME CONSUMING

# Bias and variance decomposition
# Convert all dataframe to array first
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

Average expected loss: 0.242
Average bias: 0.219
Average variance: 0.126
Sklearn 0-1 loss: 0.214


## 3.4 KNN with ChiMerge 15 intervals

In [82]:
# Prepare data and apply SMOTE
# make test & train split
X = pima15.drop('Outcome', axis=1)
y = pima15['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima15['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima15.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [70]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

KeyError: 13.0

In [71]:
# # !!!! WARING: TIME CONSUMING

# # Bias and variance decomposition
# # Convert all dataframe to array first
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train, y_train, X_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))